In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Путь к изображению
# img_path = '/content/drive/My Drive/'

In [5]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow


def predict_image(img: np.ndarray, query: np.ndarray) -> list:

  gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  gray_query = cv.cvtColor(query, cv.COLOR_BGR2GRAY)

  sift = cv.SIFT_create()

  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(gray_img,None)
  kp2, des2 = sift.detectAndCompute(gray_query,None)

  #Поиск совпадений и сортировка совпадений
  bf = cv.BFMatcher(cv.NORM_L1, crossCheck = False)
  matches = bf.match(des1,des2)
  matches = sorted(matches, key = lambda x:x.distance)

  #Соотносим особые точки и находим матрицу гомографии
  matches = matches[:5000]

  #изображение куда будем наносит рамки найденных предметов
  img_to_show = img.copy()

  while(True):

    new1 = np.float32([kp1[m.queryIdx].pt for m in matches]) # координаты контрольной точки для 1-ого изображения
    new2 = np.float32([kp2[m.trainIdx].pt for m in matches]) # координаты контрольной точки для 2-ого изображения

    H, status = cv.findHomography(new2, new1, cv.RANSAC, 4.0)

    #Если матрица гомографии не найдена или совпадений слишком мало
    if ((H is None) or (status.sum() < 4)):
      # cv2_imshow(img_to_show)
      return

    #Будем рисовать те точки, которые нам понравились
    for index in range(len(status)-1, 0, -1):
      if (status[index, 0]):
        #Находим какой индекс имеет данная точка в общем плане
        index_train = matches[index].queryIdx
        #Находим координаты этой точки в общем плане
        x, y = kp1[index_train].pt
        cv.circle(img_to_show,(int(x),int(y)), 3,(0,255,0), -1)

    #Делаем белое изображение
    white_query = np.ones_like(gray_query) * 255

    width = query.shape[1]
    height = query.shape[0]

    corners_query = np.float32([[0, 0], [0, height-1], [width-1, height-1], [width-1, 0]]).reshape(-1, 1, 2)
    corners_query_transformed = cv.perspectiveTransform(corners_query, H)

    #Трансформируем белое
    white_transformed = cv.warpPerspective(white_query, H, (img.shape[1],img.shape[0]),flags=cv.INTER_NEAREST )

    #Инвертируем белое
    white_transformed_inv = cv.bitwise_not(white_transformed)

    base_image = img.copy()

    cv.polylines(img_to_show, [corners_query_transformed.astype(np.int32)], True,(0,255,255), 10)

    corners_q_tr = corners_query_transformed.astype(np.int32)

    #Удаляем избыточный внутренний массив
    corners_q_tr_norm = corners_q_tr.squeeze()

    for i in range(len(matches)-1, 0, -1):
      index_train = matches[i].queryIdx
      #Находим координаты этой точки в общем плане
      point_to_check = kp1[index_train].pt
      # Проверить, находится ли точка внутри четырехугольника
      is_point_in_rect = cv.pointPolygonTest(corners_q_tr, point_to_check, measureDist=False)
      # Распечатать результат
      if (is_point_in_rect == 1) or (is_point_in_rect == 0):
        matches.pop(i)

  return base_image

# Вызов функции

In [6]:
# shelf = cv.imread(img_path + '/content/train_1.jpg')
# juice_box = cv.imread(img_path + '/content/template_1.jpg')
shelf = cv.imread('content/train_1.jpg')
juice_box = cv.imread('content/template_1.jpg')
predict_image(shelf, juice_box)



[ WARN:0@49.024] global loadsave.cpp:248 findDecoder imread_('/content/train_1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@49.024] global loadsave.cpp:248 findDecoder imread_('/content/template_1.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
